<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/TPUs_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
from IPython.display import clear_output
import os
import pprint
%tensorflow_version 1.x
import tensorflow as tf

# tf.debugging.set_log_device_placement(True)

print("tensorflow version:", tf.__version__, '\n')

if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)
    
    with tf.Session(tpu_address) as session:
        devices = session.list_devices()
        
    print('TPU devices:', '\n')
    pprint.pprint(devices)

If the cell above reports an error, make sure that you have enabled TPU support in the notebook settings. (Edit menu → Notebook settings)

Now, let's try a simple computation.

In [ ]:
import numpy as np

def add_op(x, y):
    return x + y
  
x = tf.placeholder(tf.float32, [10,])
y = tf.placeholder(tf.float32, [10,])
tpu_ops = tf.contrib.tpu.rewrite(add_op, [x, y])
  
session = tf.Session(tpu_address)
try:
    print('Initializing...')
    session.run(tf.contrib.tpu.initialize_system())
    print('Running ops')
    print(session.run(tpu_ops, {x: np.arange(10), y: np.arange(10)}))
finally:
    # For now, TPU sessions must be shutdown separately from
    # closing the session.
    session.run(tf.contrib.tpu.shutdown_system())
    session.close()

## TPU FLOPs

Finally, we'll try a small test of floating point computations (floating point operations per seconds. (The units are FLOPS: floating point operations per second.)

In [ ]:
N = 4096 
print(N)
COUNT = 100
import time

def flops():
    x = tf.random_uniform([N, N])
    y = tf.random_uniform([N, N])
    def _matmul(x, y):
        return tf.tensordot(x, y, axes=[[1], [0]]), y
        
    return tf.reduce_sum(tf.contrib.tpu.repeat(COUNT, _matmul, [x, y]))
  
tpu_ops = tf.contrib.tpu.batch_parallel(flops, [], num_shards=8)
  
session = tf.Session(tpu_address)
try:
    print('Warming up...')
    session.run(tf.contrib.tpu.initialize_system())
    session.run(tpu_ops)
    print('Profiling')
    start = time.time()
    session.run(tpu_ops)
    end = time.time()
    elapsed = end - start
    print("Elapse time:", elapsed, 'TFlops: {:.2f}'.format(1e-12 * 8 * COUNT * 2*N*N*N / elapsed))
finally:
    session.run(tf.contrib.tpu.shutdown_system())
    session.close()

## Next steps

More involved examples include:
- [Shakespeare in 5 minutes with Cloud TPUs and Keras](https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/shakespeare_with_tpu_and_keras.ipynb)
- [Shakespeare in 5 minutes with Cloud TPUs via TPUEstimator](https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/shakespeare_with_tpuestimator.ipynb)
- [Fashion MNIST with Keras and TPUs](https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/fashion_mnist.ipynb)

We'll be sharing more examples of TPU use in Colab over time, so be sure to check back for additional example links, or [follow us on Twitter @GoogleColab](https://twitter.com/googlecolab).

Meanwhile, you can check out the [TPUEstimator documentation on TensorFlow.org](https://www.tensorflow.org/api_docs/python/tf/contrib/tpu/TPUEstimator). TPUEstimator is an easy way to update models to take advantage of TPU acceleration.